# **CS551 Final Project** 

Mathieu Rundström, Shenshun Yao, and Yuxiang Ma

## Utility

In [0]:
from google.colab import drive

In [14]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [0]:
class TOTrainParameter(object):
    
    def __init__(self, train_file, num_class=2, embedding_dim=100, 
                 filter_sizes="3,4,5", num_filters=100, dropout_keep_prob=0.5, 
                 l2_reg_lambda=0.0, l2_reg_constraint=3.0, batch_size=50, 
                 num_epochs=25, num_checkpoint=5, learn_rate=0.001):
        self.num_class = num_class
        self.train_file = train_file
        self.embedding_dim = embedding_dim
        self.filter_sizes = filter_sizes
        self.num_filters = num_filters
        self.dropout_keep_prob = dropout_keep_prob
        self.l2_reg_lambda = l2_reg_lambda
        self.l2_reg_constraint = l2_reg_constraint
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.num_checkpoints = num_checkpoint
        self.learn_rate = learn_rate

In [0]:
import functools
import operator


def fold_left(func, acc, xs):
    return functools.reduce(func, xs, acc)

to_trainer = TOTrainParameter(
    '/content/gdrive/My Drive/cs551final/data'
)

## Preprocessing

In [0]:
import re
import os
import collections
import numpy as np
import gensim.models
from tensorflow import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

logging.set_verbosity(logging.ERROR)

def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()


def parse_mr(positive_data_file, negative_data_file):
    """
    Loads MR polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """
    # Load data from files
    positive_examples = list(open(positive_data_file, "r", encoding='utf-8').readlines())
    positive_examples = [clean_str(s.strip()) for s in positive_examples]
    negative_examples = list(open(negative_data_file, "r", encoding='utf-8').readlines())
    negative_examples = [clean_str(s.strip()) for s in negative_examples]
    
    # Generate labels
    positive_labels = [[0, 1] for _ in positive_examples]
    negative_labels = [[1, 0] for _ in negative_examples]
    
    return positive_examples, positive_labels, negative_examples, negative_labels


def parse_sst1(dir):

    index2split = {}
    with open(dir + '/stanfordSentimentTreebank/datasetSplit.txt') as train_test_validation_split:
        for line_split in train_test_validation_split.readlines()[1:]:
            index2split[int(line_split.split(',')[0])] = int(line_split.split(',')[1])

    index2class = {}
    with open(dir + '/stanfordSentimentTreebank/sentiment_labels.txt') as sent_classes:
        for sent_class in sent_classes.readlines()[1:]:
            rate = float(sent_class.split('|')[1])
            index = int(sent_class.split('|')[0])
            if 0 <= rate <= 0.2:
                index2class[index] = [1, 0, 0, 0, 0]  # 0
            elif 0.2 < rate <= 0.4:
                index2class[index] = [0, 1, 0, 0, 0]  # 1
            elif 0.4 < rate <= 0.6:
                index2class[index] = [0, 0, 1, 0, 0]  # 2
            elif 0.6 < rate <= 0.8:
                index2class[index] = [0, 0, 0, 1, 0]  # 3
            elif 0.8 < rate <= 1.0:
                index2class[index] = [0, 0, 0, 0, 1]  # 4

    x_train = []
    y_train = []
    x_validation = []
    y_validation = []
    x_test = []
    y_test = []
    vocab = collections.defaultdict(float)
    with open(dir + '/stanfordSentimentTreebank/datasetSentences.txt') as sentences:
        for line in sentences.readlines()[1:]:
            sentence_split = line.split()
            index = int(sentence_split[0])
            text = clean_str(" ".join(sentence_split[1:]))
            if index2split[index] == 1:
                x_train.append(text)
                y_train.append(index2class[index])
                for word in set(text.split()):
                    vocab[word] += 1
            elif index2split[index] == 2:
                x_test.append(text)
                y_test.append(index2class[index])
                for word in set(text.split()):
                    vocab[word] += 1
            elif index2split[index] == 3:
                x_validation.append(text)
                y_validation.append(index2class[index])
                for word in set(text.split()):
                    vocab[word] += 1

    return x_train, y_train, x_validation, y_validation, x_test, y_test, vocab


def parse_sst2(dir):

    x_train = []
    y_train = []
    x_validation = []
    y_validation = []
    x_test = []
    y_test = []
    vocab = collections.defaultdict(float)
    with open(dir + '/stanfordSentimentTreebank/stsa.binary.dev') as validation_set:
        for validation in validation_set.readlines():
            spl = validation.split()
            text = " ".join(spl[1:])
            if int(spl[0]) == 1:
                y_validation.append([0, 1])
            else:
                y_validation.append([1, 0])
            x_validation.append(text)
            for word in set(text.split()):
                vocab[word] += 1

    with open(dir + '/stanfordSentimentTreebank/stsa.binary.test') as test_set:
        for test in test_set.readlines():
            spl = test.split()
            text = " ".join(spl[1:])
            if int(spl[0]) == 1:
                y_test.append([0, 1])
            else:
                y_test.append([1, 0])
            x_test.append(text)
            for word in set(text.split()):
                vocab[word] += 1

    with open(dir + '/stanfordSentimentTreebank/stsa.binary.train') as train_set:
        for train in train_set.readlines():
            spl = train.split()
            text = clean_str(" ".join(spl[1:]))
            if int(spl[0]) == 1:
                y_train.append([0, 1])
            else:
                y_train.append([1, 0])
            x_train.append(text)
            for word in set(text.split()):
                vocab[word] += 1

    return x_train, y_train, x_validation, y_validation, x_test, y_test, vocab


def parse_trec(dir):

    x_train = []
    y_train = []

    x_validation = []
    y_validation = []

    x_test = []
    y_test = []

    label2class = {"ABBR": [1, 0, 0, 0, 0, 0], "ENTY": [0, 1, 0, 0, 0, 0], "DESC": [0, 0, 1, 0, 0, 0],
                   "HUM":  [0, 0, 0, 1, 0, 0], "LOC":  [0, 0, 0, 0, 1, 0], "NUM":  [0, 0, 0, 0, 0, 1]}

    vocab = collections.defaultdict(float)
    with open(dir + '/TREC/train_1000.label', encoding="ISO-8859-1") as test_set:
        for test in test_set.readlines():
            spl = test.split(" ")
            text = clean_str(" ".join(spl[1:]))
            y_train.append(label2class[spl[0].split(":")[0]])
            x_train.append(text)
            for word in set(text.split()):
                vocab[word] += 1

    with open(dir + '/TREC/train_2000.label', encoding="ISO-8859-1") as test_set:
        for test in test_set.readlines():
            spl = test.split(" ")
            text = clean_str(" ".join(spl[1:]))
            y_train.append(label2class[spl[0].split(":")[0]])
            x_train.append(text)
            for word in set(text.split()):
                vocab[word] += 1

    with open(dir + '/TREC/train_3000.label', encoding="ISO-8859-1") as test_set:
        for test in test_set.readlines():
            spl = test.split(" ")
            text = clean_str(" ".join(spl[1:]))
            y_train.append(label2class[spl[0].split(":")[0]])
            x_train.append(text)
            for word in set(text.split()):
                vocab[word] += 1

    with open(dir + '/TREC/train_4000.label', encoding="ISO-8859-1") as test_set:
        for test in test_set.readlines():
            spl = test.split(" ")
            text = clean_str(" ".join(spl[1:]))
            y_train.append(label2class[spl[0].split(":")[0]])
            x_train.append(text)
            for word in set(text.split()):
                vocab[word] += 1

    with open(dir + '/TREC/train_5500.label', encoding="ISO-8859-1") as test_set:
        for test in test_set.readlines():
            spl = test.split(" ")
            text = clean_str(" ".join(spl[1:]))
            y_validation.append(label2class[spl[0].split(":")[0]])
            x_validation.append(text)
            for word in set(text.split()):
                vocab[word] += 1

    with open(dir + '/TREC/TREC_10.label', encoding="ISO-8859-1") as test_set:
        for test in test_set.readlines():
            spl = test.split(" ")
            text = clean_str(" ".join(spl[1:]))
            y_test.append(label2class[spl[0].split(":")[0]])
            x_test.append(text)
            for word in set(text.split()):
                vocab[word] += 1

    return x_train, y_train, x_validation, y_validation, x_test, y_test, vocab


def transform_google_w2v(x_text_vec, embd_dim, dir):

    w2v = gensim.models.KeyedVectors.load_word2vec_format(dir + '/W2V/GoogleNews-vectors-negative300.bin', binary=True)
    vocab = []
    embedding = []
    curr = 0
    s = set((" ".join(x_text_vec)).split())
    
    for k, v in w2v.vocab.items():
        if k in s:
            vocab.append(str(k))
            embedding.append(list(w2v[k])[:embd_dim])
            curr = curr + 1
    vocab_lookup = set(vocab)
    
    for text in x_text_vec:
        for word in text.split():
            if word not in vocab_lookup:
                vocab.append(word)
                embedding.append(list(np.random.uniform(-0.25, 0.25, embd_dim)))
                curr = curr + 1

    return vocab, embedding


def transform_random_w2v(x_text_vec):
    
    vocab = []
    embedding = []
    curr = 0
    for text in x_text_vec:
        for word in text.split():
            vocab.append(word)
            embedding.append(list(np.random.uniform(-0.25, 0.25, embd_dim)))
            curr = curr + 1

    return vocab, embedding


def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]


## CNN Class Model

In [0]:
import tensorflow as tf


class InvalidInputException(Exception): 
    """傻逼Gunter的垃圾Class"""
    pass


class CNNText:
    def __init__(self, sequence_length, num_classes, vocab_size, embedding_size, embedding_vec, filter_sizes, num_filters, l2_reg_lambda=0.0, l2_reg_constrains=3, is_static=1):

        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        # Embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            
            if is_static == 1: 
                num_channel = 1
                self.W = tf.Variable(
                    tf.random_uniform([vocab_size, embedding_size], -0.25, 0.25),
                    name="W", 
                    trainable=True
                )
            elif is_static == 2 or is_static == 3:
                num_channel = 1
                self.W = tf.Variable(
                    embedding_vec,
                    name="W", 
                    validate_shape = False, 
                    trainable=(is_static == 3)
                )
                
            else: 
                num_channel = 2
                self.W = tf.Variable(
                    embedding_vec,
                    name="W", 
                    validate_shape = False, 
                    trainable=True
                )
                self.W_static = tf.Variable(
                    embedding_vec,
                    name="W_static", 
                    validate_shape = False, 
                    trainable=False
                )
            
            self.embedded_chars = tf.nn.embedding_lookup(self.W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)
            
            if is_static == 4: 
                
                self.embedded_chars_static = tf.nn.embedding_lookup(self.W_static, self.input_x)
                self.embedded_chars_expanded_static = tf.expand_dims(self.embedded_chars_static, -1)
                self.embedded_chars_expanded = tf.concat(
                    [
                        self.embedded_chars_expanded, 
                        self.embedded_chars_expanded_static
                    ], 
                    axis=-1
                )
                

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, num_channel, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                    
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv"
                )
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool"
                )
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(pooled_outputs, 3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer()
            )
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # Calculate mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")


## Train and Evaluation

In [0]:
import numpy as np
import time
import os
import datetime
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
import gensim


def train(x_train, y_train, x_validation, y_validation, x_test, y_test, to_train_param, dir_name, is_static=1):

    # Build vocabulary
    whole_doc = x_train + x_validation
    max_document_length = max([len(x.split(" ")) for x in whole_doc])
    
    print("Generating Word2Vec--")
    vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(max_document_length)
    if is_static == 1: 
        pretrain_rand = vocab_processor.fit(fold_left(operator.add, [], [x.split(" ") for x in whole_doc]))
        embedding = [[]]
    elif is_static == 2 or is_static == 3 or is_static == 4: 
        vocab, embedding = transform_google_w2v(whole_doc, to_train_param.embedding_dim, to_train_param.train_file)
        pretrain = vocab_processor.fit(vocab)
    else: 
        raise InvalidInputException("Is static should be from 1-3.")
    
    print("Transform")
    x_vec_index_train = np.array(list(vocab_processor.transform(x_train)))
    x_vec_index_validation = np.array(list(vocab_processor.transform(x_validation)))
    x_vec_index_test = np.array(list(vocab_processor.transform(x_test)))
    
    y_train = np.array(y_train)
    y_validation = np.array(y_validation)
    y_test = y_test
    print(len(y_test))
    print(len(x_vec_index_test))
    print("Start Session--")
    graph = tf.Graph()
    with graph.as_default():
        session_conf = tf.ConfigProto(
            allow_soft_placement=True,
            log_device_placement=False
        )
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            cnn = CNNText(
                sequence_length=x_vec_index_train.shape[1],
                num_classes=y_train.shape[1],
                vocab_size=len(vocab_processor.vocabulary_),
                embedding_size=to_train_param.embedding_dim,
                embedding_vec=embedding, 
                filter_sizes=list(map(int, to_train_param.filter_sizes.split(","))),
                num_filters=to_train_param.num_filters,
                l2_reg_lambda=to_train_param.l2_reg_lambda,
                is_static=is_static,
            )

            # Define Training procedure
            global_step = tf.Variable(0, name="global_step", trainable=False)
            
            # learn_rate = tf.train.exponential_decay(to_train_param.learn_rate, global_step, 100, 0.001, staircase=True)
            optimizer = tf.train.AdadeltaOptimizer(to_train_param.learn_rate, rho=0.95, epsilon=1e-06)
            # optimizer = tf.train.AdamOptimizer(to_train_param.learn_rate)
            grads_and_vars = optimizer.compute_gradients(cnn.loss)
            
            # Gradient clipping by norm
            capped_grads_and_vars = [(tf.clip_by_norm(gv[0], to_train_param.l2_reg_constraint, axes=[0]), gv[1]) for gv in grads_and_vars]
            train_op = optimizer.apply_gradients(capped_grads_and_vars, global_step=global_step)
            
            # Keep track of gradient values and sparsity (optional)
            grad_summaries = []
            for g, v in grads_and_vars:
                if g is not None:
                    grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                    sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                    grad_summaries.append(grad_hist_summary)
                    grad_summaries.append(sparsity_summary)
            grad_summaries_merged = tf.summary.merge(grad_summaries)
            
            # Output directory for models and summaries
            timestamp = str(int(time.time()))
            out_dir = os.path.abspath(os.path.join('/content/gdrive/My Drive/cs551final', dir_name, timestamp))
            print("Writing to {}\n".format(out_dir))

            # Summaries for loss and accuracy
            loss_summary = tf.summary.scalar("loss", cnn.loss)
            acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

            # Train Summaries
            train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
            train_summary_dir = os.path.join(out_dir, "summaries", "train")
            train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

            # Dev summaries
            dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
            dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
            dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)
            # Initialize all variables
            sess.run(tf.global_variables_initializer())
            # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
            checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
            checkpoint_prefix = os.path.join(checkpoint_dir, "model.ckpt")
            if not os.path.exists(checkpoint_dir):
                os.makedirs(checkpoint_dir)
            saver = tf.train.Saver(tf.global_variables(), write_version=tf.train.SaverDef.V1)
            # saver = tf.train.Saver({'embedding/W:0': cnn.W})
            # Write vocabulary
            vocab_processor.save(os.path.join(out_dir, "vocab"))
            
            vocab_dict = {v: k for k, v in vocab_processor.vocabulary_._mapping.items()}
            with open(os.path.join(checkpoint_dir, 'embeddings.tsv'), 'w') as file_1:
                for i in range(len(vocab_processor.vocabulary_)):
                    word = vocab_dict[i]
                    file_1.write('%s\n' % (word))             
            
            config = projector.ProjectorConfig()
            embed = config.embeddings.add()
            embed.tensor_name = "embedding/W:0"
            embed.metadata_path = os.path.join(checkpoint_dir, 'embeddings.tsv')
            ckpt_writer = tf.summary.FileWriter(checkpoint_dir + '/emb_viz_%s.log' % 1, sess.graph)
            projector.visualize_embeddings(ckpt_writer, config)
             
            
            

            def train_step(x_batch, y_batch, num):
                """
                A single training step
                """
                feed_dict = {
                    cnn.input_x: x_batch,
                    cnn.input_y: y_batch,
                    cnn.dropout_keep_prob: to_train_param.dropout_keep_prob
                }
                _, step, summaries, loss, accuracy = sess.run(
                    [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                    feed_dict
                )
                time_str = datetime.datetime.now().isoformat()
                if num % (int((len(y_train)-1)/to_train_param.batch_size) + 1) == 0: 
                    print("\nTrain:" + str(num / (int((len(y_train)-1)/to_train_param.batch_size + 1))))
                    print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
                train_summary_writer.add_summary(summaries, step)

            def validation_step(x_batch, y_batch):
                """
                Evaluates model on a dev set
                """
                feed_dict = {
                    cnn.input_x: x_batch,
                    cnn.input_y: y_batch,
                    cnn.dropout_keep_prob: to_train_param.dropout_keep_prob
                }
                step, summaries, loss, accuracy = sess.run(
                    [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
                dev_summary_writer.add_summary(summaries, step)

            # Generate batches
            print("Generate Batch")
            batches = batch_iter(
                list(zip(x_vec_index_train, y_train)), 
                to_train_param.batch_size, 
                to_train_param.num_epochs
            )

            # Training loop. For each batch...
            prev = 0
            for batch in batches:
                x_batch, y_batch = zip(*batch)
                train_step(x_batch, y_batch, prev)
                current_step = tf.train.global_step(sess, global_step)
                prev = current_step
                if current_step % (int((len(y_train)-1)/to_train_param.batch_size) + 1) == 0:
                    print("\nEvaluation:" + str(current_step / (int((len(y_train)-1)/to_train_param.batch_size + 1))))
                    validation_step(x_vec_index_validation, y_validation)
                    print("")
                    path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                    print("Saved model checkpoint to {}\n".format(path))
            
            
            
            
            train_summary_writer.close()
            dev_summary_writer.close()
            ckpt_writer.close()
            # Get the placeholders from the graph by name
            input_x = graph.get_operation_by_name("input_x").outputs[0]

            dropout_keep_prob = graph.get_operation_by_name("dropout_keep_prob").outputs[0]
            
            # Reset W in fully connected layer to be scaled by dropout probability during evaluation
            W_0 = graph.get_tensor_by_name("W:0")
            W_0 = tf.assign(W_0, tf.math.multiply(tf.constant(to_train_param.dropout_keep_prob, dtype=tf.float32), W_0))
            
            # Tensors we want to evaluate
            predictions = graph.get_operation_by_name("output/predictions").outputs[0]

            # Generate batches for one epoch
            batches = batch_iter(list(x_vec_index_test), 50, 1, shuffle=False)

            # Collect the predictions here
            all_predictions = []

            for x_test_batch in batches:
                batch_predictions = sess.run(predictions, {input_x: x_test_batch, dropout_keep_prob: 1.0})
                all_predictions = np.concatenate([all_predictions, batch_predictions])
    
    print(len(all_predictions))
    
    # Print accuracy if y_test is defined
    if y_test is not None:
        correct_predictions = 0
        for r in range(len(y_test)): 
            this_pred = [0 for _ in range(y_train.shape[1])]
            index = int(all_predictions[r])
            this_pred[index] = 1
            if y_test[r] == this_pred: 
                correct_predictions = correct_predictions + 1
        print("Total number of test examples: {}".format(len(y_test)))
        print("Accuracy: {:g}".format(correct_predictions/float(len(y_test))))


## Before Running your section, do a "Runtime" -> "Run before" on previous cells. 

## SST-1

### SST1 Adadelta Optimizer

SST1 Multi Channel

In [0]:
to_trainer.num_class = 5
to_trainer.num_epochs = 5
to_trainer.batch_size = 50
to_trainer.num_filters = 100
to_trainer.embedding_dim = 300
to_trainer.l2_reg_lambda = 0
to_trainer.learn_rate = 1
to_trainer.l2_reg_constraint = 3.0

x_train, y_train, x_validation, y_validation, x_test, y_test, vocab = parse_sst1(to_trainer.train_file)

train(x_train, y_train, x_validation, y_validation, x_test, y_test, to_trainer, "SST-1 Multi Channel" , is_static=4)

SST1 non static

In [0]:
to_trainer.num_class = 5
to_trainer.num_epochs = 2
to_trainer.batch_size = 50
to_trainer.num_filters = 100
to_trainer.embedding_dim = 300
to_trainer.l2_reg_lambda = 0
to_trainer.learn_rate = 1
to_trainer.l2_reg_constraint = 3.0

x_train, y_train, x_validation, y_validation, x_test, y_test, vocab = parse_sst1(to_trainer.train_file)

train(x_train, y_train, x_validation, y_validation, x_test, y_test, to_trainer, "SST-1 Non-static" , is_static=3)

SST1 static

In [0]:
to_trainer.num_class = 5
to_trainer.num_epochs = 25
to_trainer.batch_size = 50
to_trainer.num_filters = 100
to_trainer.embedding_dim = 300
to_trainer.l2_reg_lambda = 0
to_trainer.learn_rate = 1
to_trainer.l2_reg_constraint = 3.0

x_train, y_train, x_validation, y_validation, x_test, y_test, vocab = parse_sst1(to_trainer.train_file)

train(x_train, y_train, x_validation, y_validation, x_test, y_test, to_trainer, "SST-1 Static" , is_static=2)

SST1 random

In [0]:
to_trainer.num_class = 5
to_trainer.num_epochs = 25
to_trainer.batch_size = 50
to_trainer.num_filters = 100
to_trainer.embedding_dim = 300
to_trainer.l2_reg_lambda = 0
to_trainer.learn_rate = 1
to_trainer.l2_reg_constraint = 3.0

x_train, y_train, x_validation, y_validation, x_test, y_test, vocab = parse_sst1(to_trainer.train_file)

train(x_train, y_train, x_validation, y_validation, x_test, y_test, to_trainer, "SST-1 Random" , is_static=1)

### SST1 Adam Optimizer
change section Train and Evaluation, ```train``` function 
```
optimizer = tf.train.AdadeltaOptimizer(to_train_param.learn_rate, rho=0.95, epsilon=1e-06)
```
to 
```
optimizer = tf.train.AdamOptimizer(to_train_param.learn_rate)
```
to use Adam Optimizer instead of Adadelta Optimizer

SST1 Random 

In [0]:
to_trainer.num_class = 5
to_trainer.num_epochs = 5
to_trainer.batch_size = 50
to_trainer.num_filters = 100
to_trainer.embedding_dim = 300
to_trainer.l2_reg_lambda = 0
to_trainer.learn_rate = 0.001
to_trainer.l2_reg_constraint = 3.0

x_train, y_train, x_validation, y_validation, x_test, y_test, vocab = parse_sst1(to_trainer.train_file)

train(x_train, y_train, x_validation, y_validation, x_test, y_test, to_trainer, "SST-1 Random Adam" , is_static=1)

SST1 Static

In [0]:
to_trainer.num_class = 5
to_trainer.num_epochs = 5
to_trainer.batch_size = 50
to_trainer.num_filters = 100
to_trainer.embedding_dim = 300
to_trainer.l2_reg_lambda = 0
to_trainer.learn_rate = 0.001
to_trainer.l2_reg_constraint = 3.0

x_train, y_train, x_validation, y_validation, x_test, y_test, vocab = parse_sst1(to_trainer.train_file)

train(x_train, y_train, x_validation, y_validation, x_test, y_test, to_trainer, "SST-1 Static Adam" , is_static=2)

SST1 Non Static

In [0]:
to_trainer.num_class = 5
to_trainer.num_epochs = 5
to_trainer.batch_size = 50
to_trainer.num_filters = 100
to_trainer.embedding_dim = 300
to_trainer.l2_reg_lambda = 0
to_trainer.learn_rate = 0.001
to_trainer.l2_reg_constraint = 3.0

x_train, y_train, x_validation, y_validation, x_test, y_test, vocab = parse_sst1(to_trainer.train_file)

train(x_train, y_train, x_validation, y_validation, x_test, y_test, to_trainer, "SST-1 Static Adam" , is_static=3)

SST1 Multi Channel

In [0]:
to_trainer.num_class = 5
to_trainer.num_epochs = 5
to_trainer.batch_size = 50
to_trainer.num_filters = 100
to_trainer.embedding_dim = 300
to_trainer.l2_reg_lambda = 0
to_trainer.learn_rate = 0.001
to_trainer.l2_reg_constraint = 3.0

x_train, y_train, x_validation, y_validation, x_test, y_test, vocab = parse_sst1(to_trainer.train_file)

train(x_train, y_train, x_validation, y_validation, x_test, y_test, to_trainer, "SST-1 Multi Channel Adam" , is_static=4)

## SST-2

SST2 Multi Channel

In [0]:
to_trainer.num_class = 2
to_trainer.num_epochs = 25
to_trainer.batch_size = 50
to_trainer.num_filters = 100
to_trainer.embedding_dim = 300
to_trainer.l2_reg_lambda = 0
to_trainer.learn_rate = 1
to_trainer.l2_reg_constraint = 3.0

x_train, y_train, x_validation, y_validation, x_test, y_test, vocab = parse_sst2(to_trainer.train_file)

train(x_train, y_train, x_validation, y_validation, x_test, y_test, to_trainer, "SST-2 multi-channel" , is_static=4)

SST2 Non static

In [0]:
to_trainer.num_class = 2
to_trainer.num_epochs = 25
to_trainer.batch_size = 50
to_trainer.num_filters = 100
to_trainer.embedding_dim = 300
to_trainer.l2_reg_lambda = 0
to_trainer.learn_rate = 1
to_trainer.l2_reg_constraint = 3.0

x_train, y_train, x_validation, y_validation, x_test, y_test, vocab = parse_sst2(to_trainer.train_file)

train(x_train, y_train, x_validation, y_validation, x_test, y_test, to_trainer, "SST-2 non-static" , is_static=3)

SST2 Static 

In [0]:
to_trainer.num_class = 2
to_trainer.num_epochs = 25
to_trainer.batch_size = 50
to_trainer.num_filters = 100
to_trainer.embedding_dim = 100
to_trainer.l2_reg_lambda = 0
to_trainer.learn_rate = 1
to_trainer.l2_reg_constraint = 3.0

x_train, y_train, x_validation, y_validation, x_test, y_test, vocab = parse_sst2(to_trainer.train_file)

train(x_train, y_train, x_validation, y_validation,  x_test, y_test, to_trainer, "SST-2 static" , is_static=2)

SST2 Random

In [0]:
to_trainer.num_class = 2
to_trainer.num_epochs = 25
to_trainer.batch_size = 50
to_trainer.num_filters = 100
to_trainer.embedding_dim = 300
to_trainer.l2_reg_lambda = 0
to_trainer.learn_rate = 1
to_trainer.l2_reg_constraint = 3.0

x_train, y_train, x_validation, y_validation, x_test, y_test, vocab = parse_sst2(to_trainer.train_file)

train(x_train, y_train, x_validation, y_validation, x_test, y_test, to_trainer, "SST-2 rand" , is_static=1)

##TREC

TREC Random

In [0]:
to_trainer.num_class = 2
to_trainer.num_epochs = 25
to_trainer.batch_size = 50
to_trainer.num_filters = 100
to_trainer.embedding_dim = 300
to_trainer.l2_reg_lambda = 0.0
to_trainer.learn_rate = 1
to_trainer.l2_reg_constraint = 3.0

x_train, y_train, x_validation, y_validation, x_test, y_test, vocab = parse_trec(to_trainer.train_file)

train(x_train, y_train, x_validation, y_validation, x_test, y_test, to_trainer, "TREC Random", is_static=1)

TREC Static 

In [0]:
to_trainer.num_class = 2
to_trainer.num_epochs = 25
to_trainer.batch_size = 50
to_trainer.num_filters = 100
to_trainer.embedding_dim = 300
to_trainer.l2_reg_lambda = 0
to_trainer.learn_rate = 1
to_trainer.l2_reg_constraint = 3.0

x_train, y_train, x_validation, y_validation, x_test, y_test, vocab = parse_trec(to_trainer.train_file)

train(x_train, y_train, x_validation, y_validation, x_test, y_test, to_trainer, "TREC", is_static=2)

TREC Non-Static

In [0]:
to_trainer.num_class = 2
to_trainer.num_epochs = 25
to_trainer.batch_size = 50
to_trainer.num_filters = 100
to_trainer.embedding_dim = 300
to_trainer.l2_reg_lambda = 0
to_trainer.learn_rate = 1
to_trainer.l2_reg_constraint = 3.0

x_train, y_train, x_validation, y_validation, x_test, y_test, vocab = parse_trec(to_trainer.train_file)

train(x_train, y_train, x_validation, y_validation, x_test, y_test, to_trainer, "TREC", is_static=3)

TREC Multi-Channel

In [0]:
to_trainer.num_class = 2
to_trainer.num_epochs = 25
to_trainer.batch_size = 50
to_trainer.num_filters = 100
to_trainer.embedding_dim = 300
to_trainer.l2_reg_lambda = 0
to_trainer.learn_rate = 1
to_trainer.l2_reg_constraint = 3.0

x_train, y_train, x_validation, y_validation, x_test, y_test, vocab = parse_trec(to_trainer.train_file)

train(x_train, y_train, x_validation, y_validation, x_test, y_test, to_trainer, "TREC", is_static=4)